In [855]:
import yfinance as yf
import pandas as pd

# 股票列表: 半導體上市公司 （30）
tickers = ['2302.TW', '2303.TW', '2329.TW', '2330.TW', '2337.TW', '2338.TW', '2340.TW', 
          '2342.TW', '2344.TW', '2351.TW', '2363.TW', '2369.TW', '2379.TW', '2388.TW', 
          '2401.TW', '2408.TW', '2434.TW', '2436.TW', '2441.TW', '2449.TW', '2451.TW', 
          '2454.TW', '2458.TW', '2481.TW', '3006.TW', '3014.TW', '3016.TW', '3034.TW', 
          '3035.TW', '3041.TW']

# 下載數據
data = yf.download(tickers, start="2024-01-01", end="2024-12-01")

# 計算日收益率
returns = data['Adj Close'].pct_change()

# 平均日收益率和標準差
mean_returns = returns.mean()
std_dev_returns = returns.std()

# 計算相關係數
correlation = returns.corr()


[*********************100%***********************]  30 of 30 completed


In [856]:
import numpy as np

def initialize_population(size, num_assets):
    # 隨機生成初始族群
    population = np.random.randint(2, size=(size, num_assets))
    return population


In [857]:
def fitness(population, mean_returns, std_dev_returns, risk_free_rate=0.01):
    portfolio_returns = np.dot(population, mean_returns)
    portfolio_risks = np.sqrt(np.dot((population**2), (std_dev_returns**2)))
    sharpe_ratios = (portfolio_returns - risk_free_rate) / portfolio_risks
    exp_fitness_scores = np.exp(sharpe_ratios - np.max(sharpe_ratios))
    # probabilities = exp_fitness_scores / np.sum(exp_fitness_scores)
    return exp_fitness_scores


In [858]:
# def selection(population, fitness_scores):
#     # 輪盤賭選擇
#     probabilities = fitness_scores / np.sum(fitness_scores)
#     selected_indices = np.random.choice(np.arange(len(population)), size=len(population), p=probabilities)
#     return population[selected_indices]

# 加入精英保留機制
def selection(population, fitness_scores, elite_size=5):
    # 保留最優的elite_size個個體
    elite_indices = np.argsort(fitness_scores)[-elite_size:]
    elites = population[elite_indices]

    # 剩下的個體通過輪盤賭選擇
    non_elite_population = np.delete(population, elite_indices, axis=0)
    non_elite_scores = np.delete(fitness_scores, elite_indices)
    probabilities = non_elite_scores / non_elite_scores.sum()
    selected_indices = np.random.choice(np.arange(len(non_elite_population)), size=len(population)-elite_size, p=probabilities)
    selected_population = non_elite_population[selected_indices]

    # 將精英個體和選擇出的個體合併成新的族群
    new_population = np.vstack((elites, selected_population))
    return new_population


def crossover(population, crossover_rate=0.8):
    # 單點交叉
    offspring = []
    for i in range(0, len(population), 2):
        if i+1 < len(population) and np.random.rand() < crossover_rate:
            cross_point = np.random.randint(1, len(population[0]))
            offspring.append(np.concatenate([population[i][:cross_point], population[i+1][cross_point:]]))
            offspring.append(np.concatenate([population[i+1][:cross_point], population[i][cross_point:]]))
        else:
            offspring.extend([population[i], population[i+1]])
    return np.array(offspring)

def mutation(population, mutation_rate=0.02):
    # 點突變
    for individual in population:
        if np.random.rand() < mutation_rate:
            mutation_point = np.random.randint(len(individual))
            individual[mutation_point] = 1 - individual[mutation_point]
    return population


In [859]:
population_size = 100 # 每一代中將有多少個個體（投資組合）存在。
num_generations = 50 # 迭代次數，即算法將運行多少代。每一代包括選擇、交叉和突變步驟。
population = initialize_population(population_size, len(tickers))
elite_size = 5

for _ in range(num_generations):
    fitness_scores = fitness(population, mean_returns.values, std_dev_returns.values)
    population = selection(population, fitness_scores, elite_size=elite_size)
    population = crossover(population)
    population = mutation(population)


In [860]:
best_index = np.argmax(fitness(population, mean_returns.values, std_dev_returns.values))
best_portfolio = population[best_index]
best_return, best_risk = mean_returns.values[best_portfolio > 0], std_dev_returns.values[best_portfolio > 0]

In [861]:
# 最優投資組合股票: ['2330.TW', '2408.TW', '2454.TW']
# 預期回報率: [0.0007, 0.0005, 0.0011]  # 每支股票的平均日收益率
# 風險(標準差): [0.018, 0.021, 0.025]  # 每支股票的日收益率標準差

best_tickers  = [tickers[i] for i in range(len(tickers)) if best_portfolio[i] > 0]
print('最優投資組合股票:',best_tickers)
print('預期回報率:', best_return)
print('風險(標準差):', best_risk)

最優投資組合股票: ['2302.TW', '2303.TW', '2329.TW', '2330.TW', '2342.TW', '2351.TW', '2363.TW', '2379.TW', '2388.TW', '2434.TW', '2449.TW', '2451.TW', '2454.TW', '2458.TW', '3014.TW', '3034.TW']
預期回報率: [ 6.37768101e-04 -4.35024343e-04 -1.33935453e-03  2.67503395e-03
  1.85978252e-04  1.62105474e-04  9.30013196e-04  5.46973306e-04
 -1.32629123e-03 -3.63352624e-04  2.45609350e-03  9.99670525e-04
  1.71233248e-03  3.12704233e-05  1.27764354e-04  1.80462715e-04]
風險(標準差): [0.02196679 0.01543642 0.03095189 0.02217733 0.01970294 0.03263057
 0.0363808  0.02142308 0.03467    0.01171129 0.03086553 0.01834425
 0.0271918  0.0204228  0.02080876 0.01854918]


In [862]:
df = pd.DataFrame({'最優投組': best_tickers, '預期回報率/天': best_return, '風險（標準差）': best_risk})

# 篩選出預期回報率最高的 n 支股票 
n = 8
df = df.sort_values(by='預期回報率/天', ascending=False).head(n)
df

,最優投組,預期回報率/天,風險（標準差）
3,2330.TW,0.002675,0.022177
10,2449.TW,0.002456,0.030866
12,2454.TW,0.001712,0.027192
11,2451.TW,0.001000,0.018344
6,2363.TW,0.000930,0.036381
0,2302.TW,0.000638,0.021967
7,2379.TW,0.000547,0.021423
4,2342.TW,0.000186,0.019703
